In [1]:
import pandas as pd
import pandas_datareader as pdr
from bs4 import BeautifulSoup
import requests
import json
import re
import numpy as np
from nltk.corpus import stopwords

C:\Users\Lincoln\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


# prepare codemap

In [2]:
# 종목 타입에 따라 download url이 다름. 종목코드 뒤에 .KS .KQ등이 입력되어야해서 Download Link 구분 필요
stock_type = {
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt'
    }

# download url 조합
def get_download_stock(market_type=None):
    market_type = stock_type[market_type]
    download_link = 'http://kind.krx.co.kr/corpgeneral/corpList.do'
    download_link = download_link + '?method=download'
    download_link = download_link + '&marketType=' + market_type
    df = pd.read_html(download_link, header=0)[0]
    return df;
# kospi 종목코드 목록 다운로드
def get_download_kospi():
    df = get_download_stock('kospi')
    df.종목코드 = df.종목코드.map('{:06d}.KS'.format)
    return df
# kosdaq 종목코드 목록 다운로드
def get_download_kosdaq():
    df = get_download_stock('kosdaq')
    df.종목코드 = df.종목코드.map('{:06d}.KQ'.format)
    return df
# kospi, kosdaq 종목코드 각각 다운로드
kospi_df = get_download_kospi()
kosdaq_df = get_download_kosdaq()
# data frame merge
code_df = pd.concat([kospi_df, kosdaq_df])
# data frame정리
code_df = code_df[['회사명', '종목코드']]
# data frame title 변경 '회사명' = name, 종목코드 = 'code'
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

# read headlines(daum news)

In [3]:
p_dict = ['↑', '급상승', '상승', '올랐다', '오름세', '호재', '안착', '대박', '기회', '잡았나', '출발', '반등', '사자', '급등', '승부', '증가', '최고치', '안정적', '등재', 'buy', '개선']
n_dict = ['↓', '상장폐지', '뚝', '관리종목', '주의보', '낚시', '우려', '둔화', '공포', '사임', '어렵다', '어려운 ', '썰렁', '손실', '적자', '과징금', '하락', '약화', '애로']
stopword = ['이다', '너무', '은', '는', '이', '가', '을', '를', '에', '게', '도', '行', '"', "'", '.', '[', ']', ',']

In [4]:
def get_news(date_list):
    news = pd.DataFrame()
    for date in date_list:
        headline = []
        for page in range(1, 1000):
            url = f'https://news.daum.net/breakingnews/economic?page={page}&regDate={date}'
            html = requests.get(url).text
            soup = BeautifulSoup(html, 'html.parser')

            if len(soup.findAll("a", href=re.compile("https://v.daum.net/v/"))) is 0:
                break
            else:
                for index, link in enumerate(soup.findAll("a", href=re.compile("https://v.daum.net/v/"))):
                    if (link.text != None):
                        headline.append(link.text)

        headlines = set(list(headline))
        df = pd.DataFrame(headlines, columns=['headline'])
        df['date'] = date
        news = pd.concat([news, df])
    return news

In [5]:
def p_scoring(text):
    score = 0
    for keyword in p_dict:
        if keyword in text:
            score += 1
    return score

def n_scoring(text):
    score = 0
    for keyword in n_dict:
        if keyword in text:
            score += 1
    return score

In [6]:
def sw(text):
    for i in stopword:
        if i in text:
            text = text.replace(i, ' ')
    return text

In [7]:
# 회사명으로 주식 종목 코드를 획득할 수 있도록 하는 함수
def get_code(df, name):
    try:
        if len(df.query(f"name=='{name}'")['code']) == 0:
            return False
        else:
            code = df.query(f"name=='{name}'")['code'].to_string(index=False)
            # 위와같이 code명을 가져오면 앞에 공백이 붙어있는 상황이 발생하여 앞뒤로 sript() 하여 공백 제거
            code = code.strip()
            return code
    except:
        print(name)

In [8]:
def is_code(text):
    tokenization = text.split(' ')
    codes = []
    for token in tokenization:
        if len(token) != 0:
            if get_code(code_df, token.strip()):
                codes.append(get_code(code_df, token))
    if len(codes) == 0:
        return False
    else: return codes

In [9]:
news = get_news(['20200214', '20200215'])
news['p_score'] = news.headline.apply(p_scoring)
news['n_score'] = news.headline.apply(n_scoring)

In [10]:
news['text'] = news.headline.apply(sw)

In [11]:
news['is_code'] = news.text.apply(is_code)

In [12]:
news.query('is_code != False')

,headline,date,p_score,n_score,text,is_code
2,"기산텔레콤, +3.42% 상승폭 확대",20200214,1,0,기산텔레콤 +3 42% 상승폭 확대,[035460.KQ]
3,뚜레쥬르 '구움케이크'·'띠띠뽀'.. 'iF 디자인 어워드' 패키징 2관왕,20200214,0,0,뚜레쥬르 구움케 크 · 띠띠뽀 iF 디자인 어워드 패키징 2관왕,[227100.KQ]
4,"한창제지, +1.45% 상승폭 확대",20200214,1,0,한창제지 +1 45% 상승폭 확대,[009460.KS]
7,"바른손, 전일대비 20.49% 상승중.. 최근 주가 상승흐름 유지",20200214,1,0,바른손 전일대비 20 49% 상승중 최근 주 상승흐름 유지,[018700.KQ]
10,[인사] S-Oil,20200214,0,0,인사 S-Oil,[010950.KS]
...,...,...,...,...,...,...
408,"[김호준의 中企탐구] ""일본이 돌아온다""..'국가전략특구'로 부흥 노리는 日",20200215,0,0,김호준의 中企탐구 일본 돌아온다 국 전략특구 로 부흥 노리 日,[003930.KS]
427,세계 최장 3km 교량에 재료 공급 포스코..탄탄한 기본기가 '힘',20200215,0,0,세계 최장 3km 교량 재료 공급 포스코 탄탄한 기본기 힘,[005490.KS]
444,"신원, '비키(viki)' 봄·여름(S/S) 화보 공개",20200215,0,0,신원 비키(viki) 봄·여름(S/S) 화보 공개,[009270.KS]
459,"현대모비스, 정의선 대표이사 재선임.. ""미래차 시대 경쟁력 확보""",20200215,0,0,현대모비스 정의선 대표 사 재선임 미래차 시대 경쟁력 확보,[012330.KS]


In [33]:
import datetime
 

def change_rate(stock, date):
    day = datetime.date(int(date[:4]), int(date[4:6]), int(date[6:]))
    yesterday = day - datetime.timedelta(days=1)
       
    df = pdr.get_data_yahoo(stock, str(yesterday), str(day))
    
    return df

In [35]:
df = change_rate('018700.KQ', '20200213')


KeyError: 'Date'

In [282]:
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-02-14,2670,2635,2655,2660,430003,2660


In [54]:
date = '20200212'
day = datetime.date(int(date[:4]), int(date[4:6]), int(date[6:]))
str(day)

'2020-02-12'

In [82]:
pdr.get_data_yahoo('018700.KQ')

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-02-17,5282.819824,4658.750000,4803.879883,4934.500000,1869915.0,4934.500000
2015-02-23,5413.439941,4919.990234,5079.629883,5021.580078,1387751.0,5021.580078
2015-02-24,5761.759766,4876.450195,4963.529785,5761.759766,3587699.0,5761.759766
2015-02-25,5761.759766,5369.899902,5587.600098,5471.490234,2406597.0,5471.490234
2015-02-26,5674.680176,5340.870117,5456.979980,5486.000000,1599941.0,5486.000000
...,...,...,...,...,...,...
2018-01-26,4165.000000,4030.000000,4075.000000,4130.000000,358119.0,4130.000000
2018-01-29,4230.000000,4080.000000,4140.000000,4120.000000,411834.0,4120.000000
2018-01-30,4145.000000,4030.000000,4115.000000,4050.000000,235161.0,4050.000000
